In [1]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'torch'

In [2]:
# Configs
DATA_PATH = "Database/img/"
SEQ_LEN = 300   # 3s @ 10ms = 300 steps
INPUT_DIM = 6
BATCH_SIZE = 32
EPOCHS = 20
LR = 1e-3

In [3]:
# Load and preprocess data
sequences = []
labels = []

for filename in os.listdir(DATA_PATH):
    if filename.endswith('.csv'):
        path = os.path.join(DATA_PATH, filename)
        df = pd.read_csv(path, header=None, dtype=str)

        df = df.astype(str).apply(lambda col: col.map(lambda x: x.strip() if isinstance(x, str) else x))
        df = df.replace(r'[^\d\.-]', '', regex=True)
        df = df.apply(pd.to_numeric, errors='coerce')
        df = df.dropna()

        if df.shape[0] >= SEQ_LEN:
            df = df.iloc[:SEQ_LEN, :]
        else:
            pad = pd.DataFrame(np.zeros((SEQ_LEN - df.shape[0], df.shape[1])))
            df = pd.concat([df, pad], ignore_index=True)

        sequences.append(df.values.astype(np.float32))
        labels.append(filename.split('_')[0])

In [4]:
# Encode labels
label_to_idx = {label: idx for idx, label in enumerate(sorted(set(labels)))}
y_indices = [label_to_idx[label] for label in labels]

In [5]:
# Convert to tensors
X = torch.tensor(np.stack(sequences))  # [N, 300, 6]
y = torch.tensor(y_indices, dtype=torch.long)

In [6]:
print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: torch.Size([105, 300, 6])
y shape: torch.Size([105])


In [7]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [8]:
# Dataset and DataLoader
class GestureDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_loader = DataLoader(GestureDataset(X_train, y_train), batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(GestureDataset(X_test, y_test), batch_size=BATCH_SIZE)




In [9]:
# LSTM model
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        return self.fc(h_n.squeeze(0))

model = LSTMClassifier(INPUT_DIM, 128, len(label_to_idx))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

LSTMClassifier(
  (lstm): LSTM(6, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=5, bias=True)
)

In [10]:
# Training
optimizer = optim.Adam(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)torch.save(model.state_dict(), "data_glove_lstm.pth")

        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {total_loss:.4f}")

Epoch 1/20 - Loss: 4.7799
Epoch 2/20 - Loss: 4.4308
Epoch 3/20 - Loss: 4.3245
Epoch 4/20 - Loss: 4.1512
Epoch 5/20 - Loss: 4.0092
Epoch 6/20 - Loss: 3.9308
Epoch 7/20 - Loss: 3.7905
Epoch 8/20 - Loss: 3.6661
Epoch 9/20 - Loss: 3.5429
Epoch 10/20 - Loss: 3.4411
Epoch 11/20 - Loss: 3.3807
Epoch 12/20 - Loss: 3.2469
Epoch 13/20 - Loss: 3.1439
Epoch 14/20 - Loss: 3.0529
Epoch 15/20 - Loss: 2.9693
Epoch 16/20 - Loss: 2.8791
Epoch 17/20 - Loss: 2.7694
Epoch 18/20 - Loss: 2.7638
Epoch 19/20 - Loss: 2.6292
Epoch 20/20 - Loss: 2.6008


In [11]:
# Evaluation
model.eval()
all_preds = []
all_targets = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X = batch_X.to(device)
        outputs = model(batch_X)
        preds = torch.argmax(outputs, dim=1).cpu()
        all_preds.extend(preds.numpy())
        all_targets.extend(batch_y.numpy())

acc = accuracy_score(all_targets, all_preds)
print(f"\nTest Accuracy: {acc:.4f}")

# Generate target names from label_to_idx
idx_to_label = {v: k for k, v in label_to_idx.items()}
target_names = [idx_to_label[i] for i in range(len(idx_to_label))]

print("\nClassification Report:\n", classification_report(all_targets, all_preds, target_names=target_names))





Test Accuracy: 0.5714

Classification Report:
               precision    recall  f1-score   support

           A       0.33      0.25      0.29         4
           B       1.00      0.25      0.40         4
           C       0.40      0.50      0.44         4
           D       0.67      0.80      0.73         5
           E       0.67      1.00      0.80         4

    accuracy                           0.57        21
   macro avg       0.61      0.56      0.53        21
weighted avg       0.62      0.57      0.54        21



In [12]:
# Predict and print results for the test set
print("\nPredictions on Test Set:")
for i in range(len(all_preds)):
    true_label = idx_to_label[all_targets[i]]
    predicted_label = idx_to_label[all_preds[i]]
    print(f"{i+1}: True = {true_label}, Predicted = {predicted_label}")


Predictions on Test Set:
1: True = E, Predicted = E
2: True = A, Predicted = C
3: True = C, Predicted = C
4: True = A, Predicted = C
5: True = D, Predicted = D
6: True = D, Predicted = D
7: True = C, Predicted = C
8: True = D, Predicted = D
9: True = D, Predicted = D
10: True = E, Predicted = E
11: True = A, Predicted = A
12: True = B, Predicted = D
13: True = E, Predicted = E
14: True = C, Predicted = A
15: True = B, Predicted = E
16: True = B, Predicted = B
17: True = B, Predicted = D
18: True = A, Predicted = C
19: True = C, Predicted = E
20: True = D, Predicted = A
21: True = E, Predicted = E


In [13]:
print("\nIncorrect Predictions:")
for i in range(len(all_preds)):
    if all_preds[i] != all_targets[i]:
        true_label = idx_to_label[all_targets[i]]
        predicted_label = idx_to_label[all_preds[i]]
        print(f"Sample {i+1}: True = {true_label}, Predicted = {predicted_label}")


Incorrect Predictions:
Sample 2: True = A, Predicted = C
Sample 4: True = A, Predicted = C
Sample 12: True = B, Predicted = D
Sample 14: True = C, Predicted = A
Sample 15: True = B, Predicted = E
Sample 17: True = B, Predicted = D
Sample 18: True = A, Predicted = C
Sample 19: True = C, Predicted = E
Sample 20: True = D, Predicted = A


In [14]:
torch.save(model.state_dict(), "data_glove_lstm.pth")